In [1]:
import tensorflow
import cv2
#Download the necessary packages and modules
from tensorflow.keras.optimizers import RMSprop
from keras.preprocessing.image import ImageDataGenerator
import cv2
from keras.models import Sequential
from keras.layers import Conv2D, Input, ZeroPadding2D, BatchNormalization, Activation, MaxPooling2D, Flatten, Dense,Dropout
from keras.models import Model, load_model
from keras.callbacks import TensorBoard, ModelCheckpoint
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.utils import shuffle
import imutils
import numpy as np


'''We must create the neural network. The two sets of Conv and MaxPool layers in this convolution network are 
used to extract features from the dataset. This is followed by a Flatten and Dropout layer to 
transform data into a 1D format and guarantee overfitting.'''

model =Sequential([
    Conv2D(100, (3,3), activation='relu', input_shape=(150, 150, 3)),
    MaxPooling2D(2,2),
    
    Conv2D(100, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    
    Flatten(),
    Dropout(0.5),
    Dense(50, activation='relu'),
    Dense(2, activation='softmax')
])
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])

TRAINING_DIR = "C:/Users/Surplus/Downloads/Dataset/train"
train_datagen = ImageDataGenerator(rescale=1.0/255,
                                   rotation_range=40,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True,
                                   fill_mode='nearest')

train_generator = train_datagen.flow_from_directory(TRAINING_DIR, 
                                                    batch_size=10, 
                                                    target_size=(150, 150))
VALIDATION_DIR = "C:/Users/Surplus/Downloads/Dataset/test"
validation_datagen = ImageDataGenerator(rescale=1.0/255)

validation_generator = validation_datagen.flow_from_directory(VALIDATION_DIR, 
                                                         batch_size=10, 
                                                         target_size=(150, 150))

#To continue saving the right model but after every training program , establish a callback checkpoint
checkpoint = ModelCheckpoint('model2-{epoch:03d}.model',monitor='val_loss',verbose=0,save_best_only=True,mode='auto')

 
history = model.fit_generator(train_generator,
                              epochs=10,
                              validation_data=validation_generator,
                              callbacks=[checkpoint])

model.save('model.h5')

Found 1315 images belonging to 2 classes.
Found 194 images belonging to 2 classes.
Epoch 1/10


C:\Users\Surplus\AppData\Local\Temp\ipykernel_13400\2439036014.py:63: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = model.fit_generator(train_generator,


132/132 [==============================] - ETA: 0s - loss: 0.6064 - acc: 0.6783

INFO:tensorflow:Assets written to: model2-001.model\assets


INFO:tensorflow:Assets written to: model2-001.model\assets


...

INFO:tensorflow:Assets written to: model2-006.model\assets


132/132 [==============================] - 47s 355ms/step - loss: 0.2443 - acc: 0.9125 - val_loss: 0.0707 - val_acc: 0.9588
Epoch 7/10
132/132 [==============================] - ETA: 0s - loss: 0.2122 - acc: 0.9224

INFO:tensorflow:Assets written to: model2-007.model\assets


INFO:tensorflow:Assets written to: model2-007.model\assets


132/132 [==============================] - 43s 324ms/step - loss: 0.2122 - acc: 0.9224 - val_loss: 0.0682 - val_acc: 0.9742
Epoch 8/10
132/132 [==============================] - 41s 313ms/step - loss: 0.2536 - acc: 0.9042 - val_loss: 0.0804 - val_acc: 0.9845
Epoch 9/10
132/132 [==============================] - 42s 314ms/step - loss: 0.2436 - acc: 0.9171 - val_loss: 0.0917 - val_acc: 0.9691
Epoch 10/10
132/132 [==============================] - 44s 335ms/step - loss: 0.1953 - acc: 0.9346 - val_loss: 0.0762 - val_acc: 0.9794


In [5]:
import cv2
import numpy as np
from tensorflow.keras.models import load_model
model=load_model("./model.h5")

labels_dict={0:'without mask',1:'mask'}
color_dict={0:(0,0,255),1:(0,255,0)}

size = 4
webcam = cv2.VideoCapture(0) #Use camera 0

#In the next code we will load the xml file
classifier = cv2.CascadeClassifier('C:/Users/Surplus/Downloads/Dataset/frontalFace10/haarcascade_frontalface_default.xml')

while True:
    (rval, im) = webcam.read()
    im=cv2.flip(im,1,1) #Flip to act as a mirror

    # Increase the image size
    mini = cv2.resize(im, (im.shape[1] // size, im.shape[0] // size))

    # detect MultiScale / faces 
    faces = classifier.detectMultiScale(mini)

    # Each face should have rectangles around it
    for f in faces:
        (x, y, w, h) = [v * size for v in f] #Scale the shapesize backup
        #Save just the rectangle faces in SubRecFaces
        face_img = im[y:y+h, x:x+w]
        resized=cv2.resize(face_img,(150,150))
        normalized=resized/255.0
        reshaped=np.reshape(normalized,(1,150,150,3))
        reshaped = np.vstack([reshaped])
        result=model.predict(reshaped)
        #print(result)
        
        label=np.argmax(result,axis=1)[0]
      
        cv2.rectangle(im,(x,y),(x+w,y+h),color_dict[label],2)
        cv2.rectangle(im,(x,y-40),(x+w,y),color_dict[label],-1)
        cv2.putText(im, labels_dict[label], (x, y-10),cv2.FONT_HERSHEY_SIMPLEX,0.8,(255,255,255),2)
        
    # Show the image
    cv2.imshow('LIVE',   im)
    key = cv2.waitKey(10)
    # To break out of the ESS key should be pressed
    if key == 27: #The Esc key
        break
# Stop video
webcam.release()

# Close all started windows
cv2.destroyAllWindows()

1/1 [==============================] - 0s 35ms/step
